# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**19/06/2020 - C12 S7**

# Redes Neuronales 

## Introducción
La redes neuronales aparecen bajo el nombre de **perceptron**, propuesta por [Rosenblatt en 1958](http://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf). Este es un modelo de clasificación consistente en pesos $w$ y una función de output del tipo $f(w^t\mathbf{x})$ para el input $\mathbf{x}$. Si bien este modelo es en esencia muy similar a la regresión logística, el percetron implementa $f(\cdot)$ como una función de salto, que entrega el valor 1 para argumentos mayores que 0 y retorna 0 en otro caso. El problema de este modelo consiste en su poca expresibilidad, pues solo permite hacer separaciones lineales. 

Sobre el modelo anterior, se construye el perceptron multicapa, el cual consiste en múltiples modelos de perceptron intecomunicadas, cada uno de estos modelos se denota como *unidad* (*unit*) y se organizan en capas secuenciales conocidas como *capa input*, corresponde a un conjunto de perceptrones que operan de manera paralela sobre un vector input. El resultado de sus clasificaciones corresponde a un vector, donde cada componente se asocia un perceptron de la capa input. Las siguientes capas se denotan como *capas ocultas* y se contruyen de manera análoga a la capa input (o inicial), la primera capa oculta consiste en un conjunto de percetrones que operan sobre el output de la capa input. Sobre esta primera capa oculta pueden haber multiples capas ocultas que operan sobre el output de la capa oculta anterior. El proceso de generar una salida sobre la entrada de una capa anterior se denota *propagación hacia adelante* (*feedforward*) la propagación termina en una última capa denominada *capa output* que entrega el resultado final de la clasificación. 

Bajo el punto de vista del aprendizaje automático, las capas ocultas de un perceptron multicapa (red neuronal) generan abstracciones o características a partir de los datos sobre los cuales operan. 

Para trabajar con redes neuronales haremos uso de la librería **Pytorch**. Esta librería consiste en un conjunto de herramientas diseñadas para generar modelos basados en redes neuronales utilizando las capacidades de computo distribuido que ofrecen las GPU (unidades de procesamiento gráfico / tarjetas de video). Esto permite operaciones de vectorización aceleradas y distribuidas. Esta librería se importa como `torch`.

**Ejemplo**

Se implementa una red neuronal simple utilizando Pytorch. Para esto, se importa el módulo y se indica una semilla aleatoria.

In [ ]:
import torch

# Se asigna un valor de reproductibilidad
torch.manual_seed(6202)

Se carga el dataset sobre el cual trabajaremos, en este caso será un conjunto de datos de vinos. Este conjunto consta de 13 atributos continuos, cada vino posee un identificador dentro de 3 posibilidades, la idea es poder clasificar cada vino para predecir su identificador. 

In [ ]:
import pandas as pd

names = [
    'class','Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
    'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
    'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline'
]

wine_data = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
    names=names)

wine_data.head()

Se procede a hacer una separación en entrenamiento y test, se hará es una codificación dummy para la variable de respuesta y se estandarizan las variables numéricas

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

response = ['class']
num_cols = wine_data.loc[:, 'Alcohol':].columns.values

data_transform = ColumnTransformer(
    transformers=[('normaliza', StandardScaler(),num_cols), 
                  ('codifica', OrdinalEncoder(), response)])

Para evitar fuga de información, se hace una separación en train y test para luego transformar los datos

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(wine_data, test_size=.2)

Se aplica el preprocesamiento sobre el conjunto train

In [ ]:
data = data_transform.fit_transform(data_train)

# Se obtienen las variables numericas y de respuesta

X_train = data[:,:-1].copy()
y_train = data[:,-1:].copy()

El perceptron multicapa a modelar consiste en 10 capas ocultas y su función de activación tiene la forma $f(x) = max(0,x)$, esta es una función conocida y se denota como ReLU (Rectified Linear Unit). Se definen los parámetros, para la función de activación se utiliza el módulo de redes neuronales de Pytorch, se puede acceder a este módulo por medio de `torch.nn`.

In [ ]:
input_dim, output_dim = (X_train.shape[1], 3)

# Equivalente al output de la capa input
input_dim_capas_ocultas = 5

f_activation = torch.nn.ReLU()

Para definir la primera capa, tenemos que tener en cuenta que buscamos una transformación de la forma $x \mapsto w^t x$ que luego es clasificada por medio de $f(w^t x)$. Esto quiere decir que los parámetros a entrenar serán los vectores de peso $w$ asociados a cada unidad (perceptron) de cada capa (input, ocultas y output). Pytorch ofrece una abstracción para capas formadas por perceptrones de la forma $f(w^t x)$. Esta abstracción es la clase `Linear` (observe que $x \mapsto w^t x$ es una transformación lineal), se procede a definir la capa input.




In [ ]:
from torch.nn import Linear

capa_input = Linear(in_features  = input_dim,
                    out_features = input_dim_capas_ocultas)

Luego se definen las capas ocultas, estas también corresponden a transformaciones lineales sobre sus capas predecesoras. En este caso, cada capa oculta tendrá dimensión 5 como input y output. Se define una capa oculta

In [ ]:
Linear(input_dim_capas_ocultas, input_dim_capas_ocultas)

La arquitectura de esta red es de naturaleza secuencial y corresponde al siguiente algoritmo:

1. Input: x de dimensión 13
2. Opera x en cada unidad de la capa lineal input, es decir, calcula $w_i^t x$ para $i = 1, \ldots, 5$ (dimensión de salida de la capa input).
3. Opera RelU($w_i^t x$) para $i = 1, \ldots, 5$, se obtiene un vector de dimensión 5. 

4. Para cada una de las 2 capas ocultas y de manera secuencial:
    1. Genera una transformación lineal sobre los outputs de la capa anterior.
    2. Aplica ReLU sobre las transformaciones lineales.
    3. En la última capa oculta el vector resultante se pasa a la capa output.

5. Se recibe el resultado de la última capa oculta, se transforma de manera lineal sobre 3 perceptrones para finalmente ser transformada por medio de una función softmax (dimensión del output es 3). 

Esto se reduce a: 
```
X  -> capa input -> ReLU() -> Co_1 -> ReLU() -> Co_2 -> ReLU() -> capa output -> predicción
```

Para programar el esquema anterior se hace uso de un diccionario ordenado `OrderedDict` de la librería `collections`. Este tipo de objetos opera de manera similar a las Pipelines de Scikit-learn, pues reciben un conjunto de tuplas del tipo `(identificador,objeto)`. Se procede a generar la arquitectura de la red:

In [ ]:
from collections import OrderedDict

n_capas_ocultas = 2

# capa input
input_ = ('capa input', capa_input)
relu = ('relu', f_activation)

steps = [input_, relu]

# capas ocultas
for i in range(n_capas_ocultas):
    
    capa = ('capa oculta_' + str(i),
         Linear(input_dim_capas_ocultas, input_dim_capas_ocultas))
    
    steps.extend([capa,relu])

# capa output
output = ('capa ouput', Linear(input_dim_capas_ocultas, output_dim))

steps.extend([(output),('Softmax',torch.nn.Softmax(dim=1))])

# Se utiliza la estructura de diccionario ordenado
steps = OrderedDict(steps)

Luego, cuando ya se posee la arquitectura, se inicializa un objeto `Sequential` del módulo `nn`, este objeto permite modelar una red neuronal multicapa recibiendo como input los componentes de la arquitectura de manera ordenada.

In [ ]:
from torch.nn import Sequential

#MLP -> multi layer perceptron
mlp = Sequential(steps)

El resultado entregado por la capa output corresponde a un vector de tres dimensiones, se asigna la clase predicha a aquella componente con el mayor valor. Luego de definir la red, es necesario definir un criterio de optimalidad (función de perdida), en este caso se utiliza la entropía cruzada. Este criterio permite comparar dos distribuciones de probabilidad $q$ (aproximación) y $p$ (real) en términos de la diferencia de información esperada (en bits por ejemplo) al utilizar la distribución $q$ para describir un eventos codificados, optimizados para $p$. Dado que se trabaja en un problema de clasificación (supervisado) se conoce la distribución real $p$ para una etiqueta  (ej: etiqueta (1,0,0), distribución (100%, 0, 0) ), por otra parte, la distribución aproximada viene dada por nuestro modelo. La entropía cruzada entre $p$ y $q$ se expresa según:
$$
H(p, q)=-\sum_{x \in \mathcal{X}} p(x) \log q(x)
$$

Se implementa mediante:

In [ ]:
criterio = torch.nn.CrossEntropyLoss()

Se debe seleccionar el optimizador a utilizar, en este caso será **descenso de gradiente estocástico** (SGD, se estudiará con más detenimiento posteriormente). Este optimizador tiene un parámetro asociado a la proporción de aprendizaje (*learning rate*) y momentum. Se inicializa entregando dichos parámetros y los coeficientes sobre los que opera, en este caso, los parámetros de la red `mpl` a los cuales se acceede por medio del método `.parameters()`

In [ ]:
optimizador = torch.optim.SGD(mlp.parameters(), lr=0.05, momentum=0.1)

Finalmente, se puede entrenar la red definida, para ello se define una cantidad de *épocas* (*epochs*), esto se refiere a la cantidad de veces que se entrena utilizando el conjunto de entrenamiento. Este proceso tiene el siguiente orden:

1. Genera un conjunto de inputs en un formato compatible.
2. En cada época:
    1. Inicializa los gradientes asociados al optimizador, esto evita que se acumulen gradientes entre épocas.
    2. Se opera sobre los inputs para obtener las predicciones.
    3. Se calcula el error de predicción.
    4. Se propagan los valores de la función de perdida para ajustar los pesos de la red y mejorar predicciones. Este proceso se denomina como *propagación hacia atrás* (*backpropagation*).
    5. Se pasa a la siguiente época.
    
Se implementa el esquema anterior:

In [ ]:
#Paso A
datos_input = torch.autograd.Variable(torch.Tensor(X_train))
labels = torch.autograd.Variable(torch.Tensor(y_train.reshape([-1,])).long())

In [ ]:
epocas = 1000
for ep in range(epocas):
    #Paso B
    optimizador.zero_grad()
    for i in range(20):
        #Paso C
        out = mlp(datos_input)
        #out.requires_grad_(True)

        #Paso D 
        loss = criterio(out, labels)
        
        #Paso E
        loss.backward() 
        optimizador.step()
    
    if ((ep+1) % 100) == 0 : print('Epoca: ', ep+1, 'Loss: ', loss.data)

Se estudia el rendimiento en train y test

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

dt_test = data_transform.transform(data_test)

X_test = dt_test[:, :-1]
y_test = dt_test[:, -1:]

f = lambda x: torch.argmax(mlp(x), dim=1)

#Train error
datos_input = torch.Tensor(X_train).float()
preds_train = f(datos_input)

print('Reporte train : \n',
      classification_report(y_train.reshape([-1,]), preds_train))

In [ ]:
# Test Error

datos_input = torch.Tensor(X_test).float()
preds_test = f(datos_input)

print('Reporte test : \n', 
      classification_report(y_test.reshape([-1,]), preds_test))

se puede decir que el clasificador entrenado fue un éxito.

Las redes neuronales pueden ser descritas como un modelo matemático de procesamiento de información. En general, una red neuronal puede considerarse como un sistema con las siguientes características:

1. El procesamiento de la información ocurre en unidades llamadas neuronas.
2. Las neuronas están conectadas e intercambian información (o señales) por medio de sus conexiones.
3. Las conexiones entre neuronas pueden ser fuertes o débiles, dependiendo de como se procesa la información.
4. Cada neurona tiene un estado interno determinado por todas las conexiones que posee.
5. Cada neurina tiene una función de activación que opera sobre su estado, esta función determina la información que se comparte a otras neuronas.

En términos operativos, una red neuronal posee una **arquitectura** que describe el conjunto de conexiones entre neurona y un proceso de **aprendizaje** asociado, que describe el proceso entrenamiento.

Las **neuronas** por tanto, pueden ser definidas por medio de la siguiente relación:
$$
y=f\left(\sum_{i} x_{i} w_{i}+b\right)
$$

Acá se hace el calculó $w^t x + b = \sum_i x_i w_i + b$ sobre los inputs $x_i$ y los pesos $w_i$. Estos últimos, son valores numéricos que representan las conexiones entre neuronas, el peso $b$ se denomina *bias*. Luego se calcula el resultado de aplicar la función de activación $f(\cdot)$. Existen distintos tipos de funciones de activación dentro de estas se pueden nombrar:

* $f(x)=x$ la función identidad.
* $f(x)=\left\{\begin{array}{l}1 \text { if } x \geq 0 \\ 0 \text { if } x<0\end{array}\right.$ la función de activación de umbral.
* $f(x)=\frac{1}{1+\exp (-x)}$ la función sigmoide logistica, es una de las más utilizadas.
* $f(x) =\frac{1-\exp (-x)}{1+\exp (-x)}$ la función sigmoide bipolar, esta corresponde a una sigmoide escalada a $(-1,1)$.
* $f(x) = \frac{1-\exp (-2 x)}{1+\exp (-2 x)}$ la tangente hiperbólica. 

* $f(x)=\left\{\begin{array}{l}x\text { if } x \geq 0 \\ 0 \text { if } x<0\end{array}\right.$ La función de activación ReLU.

**Ejercicio**

1. Existen variantes de la función de activación ReLU, investigue al menos 3 y compare sus diferencias.

Por su parte, las **capas de una red** conforman su arquitectura al agrupar conjuntos de neuronas. Acá se puede distinguir la *capa input* que representa las condiciones iniciales del dataset. La *capa output* puede tener una neurona (ej: problemas de regresión) o más (como en el ejemplo anterior). Si hay capas entre el input y el output, estas se denominan *capas ocultas*. En general las capas pueden estar completamente conectadas (como en el ejemplo implementado) o pueden faltar conexiones entre neuronas, esto se puede representar por los valores de los pesos, por ejemplo, el si denotamos $w_{ij}^{(k,k+1)}$ como los pesos entre el output de la neurona $i$ en la capa $k$ con el input de la neurona $j$ de la capa $k+1$, un peso $w_{ij}^{(k,k+1)} = 0$ implica que la neurona $j$ esima de la capa $k+1$ no posee interacción con la neurona $i$ esima de la capa $k$. 

**Ejercicios**

1. El esquema presentado en el ejemplo anterior implica que las capas de una red deben conectarse de manera secuencial. En general esto no tiene porque ser cierto, en efecto, las conexiones entre capas pueden ser modeladas por medio de grafos *acíclicos dirigidos*. ¿Por qué es necesario utilizar grafos de este tipo? 

2. El teorema de aproximación universal para redes neuronales dice que toda función continua en un compacto de $\mathbb{R}^n$ puede ser aproximada por una red neuronal con al menos una capa oculta. Para tener una intuición sobre esto, observe que es posible aproximar funciones continuas por medio de sumas ponderadas de *funciones boxcar* o *cajón*, estás, para una altura $A$ y base $(b-a)$ , $b >a$ se puede escribir como $\operatorname{boxcar}(x)=A(H(x-a)-H(x-b))$, donde:
$$
H(x)=\left\{\begin{array}{ll}0, & x<0 \\ 1, & x\geq 0\end{array}\right.
$$ 

Implemente una función boxcar utilizando una red neuronal con inputs en $x \in \mathbb{R}$, una capa oculta de dos neuronas, cada una con función de activación $f(x) = 1 /(1+\exp (-x))$, la red genera un output $y \in \mathbb{R}$.

El proceso de aprendizaje en redes pasa por definir un **esquema de entrenamiento**. Esto hace referencia a encontrar un conjunto de conexiones óptimas entre las las neuronas de la arquitectura para generar predicciones. Estas conexiones optimas se representan por los pesos de la red en cada capa y se entrenan en función de algún esquema de optimización (Descenso de gradiente por ejemplo).

Observemos por ejemplo una regresión lineal, en este caso la red consta de una capa oculta, un output unidimensional y la función de activiación identidad. Es posible entrenar una regresión lineal utilizando descenso de gradiente, para ello se define el función a minimizar $J$ como:
$$
J = \frac{1}{n} \sum_{i=0}^{n}\left(y_{i}-t_{i}\right)^{2}=\frac{1}{n} \sum_{i=0}^{n}\left(w^t x_{i} - {i}\right)^{2}
$$

En este caso $y_i$ representa el output del modelo lineal. El funcional a minimizar es el error cuadrátrico medio. El esquema de descenso de gradiente para este modelo pasa a ser:
$$
w \rightarrow w-\lambda \nabla(J(w))
$$

El cual se detiene al llegar a cierto umbral para $J$ o luego de cierto número de iteraciones. 

**Ejercicios**

1. Encuentre una expresión exacta para $\nabla(J(w))$ e implemente un esquema de descenso de gradiente para la regresión lineal. ¿Qué nombre se asocia al parámetro $\lambda$ en el esquema $w \rightarrow w-\lambda \nabla(J(w))$?¿Cuál es su función?

2. El esquema de regresión logística corresponde a una red neuronal idéntica a la utilizada para regresión lineal, con la salvedad de que posee una función de activación sigmoide logística. Calcule un descenso de gradiente para este tipo de red. 

Se debe observar que al implementar el esquema anterior, se produce una acumulación de error el todo en conjunto de entrenamiento. En conjunto de datos grandes esto no es práctico. Una solución es implementar *Descenso de Gradiente Estocástico*, este algoritmo consiste en aproximar los pesos del esquema anterior sobre un subconjunto de observaciones del conjunto de entrenamiento, este subconjunto se denota como **mini -batch**.

Las redes de una capa oculta (regresión logística y lineal) poseen reglas sencillas de actualización según la función de activación, con esto se pueden actualizar los pesos de cada conexión utilizando el error de predicción como referencia. En redes con múltiples capas sólo se puede aplicar esta técnica a las neuronas de la última capa oculta, esto pues, no se tienen valores reales (*target*) con los cuales comparar los resultados de las capas ocultas intermedias. En este caso, lo que se hace es calcular el error de la ultima capa oculta para luego hacer una **estimación** del error cometido en la capa oculta anterior, este proceso se hace de manera recurrente hasta llegar a la capa inicial. Tal proceso de denota como *propagación hacia atrás* (**backpropagation**). Este algoritmo es la base del aprendizaje con redes neuronales. Para comprenderlo es necesario definir los siguientes conceptos:


1. Se define $w_{ij}$ como el peso entre la neurona $i$ esima de la capa $l$ con la neurona $j$ esima de la capa $l+1$. Es decir, el subindice izquierdo hace referencia a una capa anterior a la que neurona que representa el subindice derecho. Al convención $l$ hace referncia a cualquier capa oculta de la red.

2. Se utiliza $y$ para denotar inputs y outputs entre capas.  Es decir, $y_i$ es el input de la capa $l+1$ pero también el output de la capa $l$. 

3. El funcional de pérdida se denota como $J$, el valor de activación $w^t x$ se denota como $a$.

4. $a_j$ es una función de activación con pesos $w_j$, $y_j$ es una función de $a_j$ y $J$ es función de $y_j$. Según la regla de la cadena se tiene así:
$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$

5. Como se sabe que $\frac{\partial a_{j}}{\partial w_{i, j}}=y_{i}$, se tiene:

$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} y_{i}
$$

6. Para las capas anteriores se cumple la misma formula (por notación):$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$
Aunque se tengan muchas capas, siempre es posible concentrarse en pares de capas consecutivas, por que se logra siempre tener una capa input y una capa output. Como se sabe que $\frac{\partial a_{j}}{\partial w_{i, j}}=y_{i}$, es decir, la derivada parcial del valor de  activación de la neurona $j$-ésima en la capa $l+1$ con respecto al peso de la conexión del output $y_j$ entre las capas $l$ y $l+1$ con la neurona $i$-ésima, es justamente el valor del output sin ser ponderado por el peso de la conexión. Más aún, $\frac{\partial y_{j}}{\partial a_{j}}$ es la derivada de la función de activación que se puede calcular. Entonces, en la expresión obtenida por la regla de la cadena, sólo hace falta calcular $\frac{\partial J}{\partial y_{j}}$, la derivada del error con respecto a la función de activación en la segunda capa. Con esto, se pueden calcular todas las derivadas partiendo desde la última capa hacia atrás pues:
    * Se puede calcular esta derivada para la última capa
    * Se tiene la formula que permite calcula la derivdada de una capa, teniendo calculada la derivada de la capa siguiente. 

**Ejemplo**

En la siguiente ecuación, $y_i$ es el output de la primera capa, $y_j$ el ouput de la segunda capa. Aplicando la regla de la cadena se tiene:
$$
\frac{\partial J}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial y_{i}}
$$

Se puede calcular $\frac{\partial y_{j}}{\partial a_{j}}$ y $\frac{\partial a_{j}}{\partial y_{i}}=w_{i, j}$ , si $\frac{\partial J}{\partial y_{j}}$ es conocido (gradiente en la capa segunda), se puede calcular $\frac{\partial J}{\partial y_{i}}$ (gradiente en la capa primera). En resumen, si se tienen 3 capas conectadas de manera secuencia según los inputs (outputs) $y_i \rightarrow y_j \rightarrow y_k$ entonces se pueden aplicar las ecuaciones:

$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$

y 

$$
\frac{\partial J}{\partial y_{j}}=\sum_{k} \frac{\partial J}{\partial y_{k}} \frac{\partial y_{k}}{\partial y_{j}}
$$

Para calcular las derivadas del costo en cada capa. 

Al definir $\delta_{j}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}}$, se tiene que $\delta_j$ representa la variación de costo con respecto a cada valor de activación. Con esto se puede reescribir:

$$
\frac{\partial J}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial y_{i}}=\sum_{j} \delta_{j} w_{i, j}
$$

Con lo que $\delta_{i}=\left(\sum_{j} \delta_{j} w_{i, j}\right) \frac{\partial y_{i}}{\partial a_{i}}$. Esto se traduce en poder calcular la variación para la siguiente capa:

$$\delta_{j}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}}$$

$$\delta_{i}=\left(\sum_{j} \delta_{j} w_{i, j}\right) \frac{\partial y_{i}}{\partial a_{i}}$$

Combinando las ecuaciones:

$$
\frac{\partial J}{\partial w_{i, j}}=\delta_{j} \frac{\partial a_{j}}{\partial w_{i, j}}=\delta_{j} y_{i}
$$

Con lo que la regla de actualizació para los pesos de cada capa pasa a ser: 

$$
w_{i, j} \rightarrow w_{i, j}-\eta \delta_{j} y_{i}
$$

En Pytorch es posible implementar reglas de actualización basadas en propagación hacia atrás de manera automática. Para ello, es necesario comprender los tipos de datos asociados a esta librería. 

En primer lugar, se tienen los **tensores**, estos son arreglos n dimensionales, soportan almacenamiento en GPU. Las operaciones y sintaxis de este tipo de objetos es muy similar a los arreglos de NumPy. 

**Ejemplo**

Pytorch se mimetiza con NumPy en cuanto a la definición de sus métodos y compatibilidad cruzada, por ejemplo, para definir un arreglo de 2x2 se puede utilizar la función `randn` (análoga a la de NumPy)

In [ ]:
x = torch.randn(2, 2)
x

Se pueden importar arreglos de numpy de manera bastante sencilla

In [ ]:
x = np.random.randn(2,2)
x = torch.from_numpy(x)
x

La función `torch.cuda.is_available()` permite detectar si existe una GPU disponble para computo. En caso afirmativo, se pueden cargar tensores en la GPU para su posterior computo, para ello, se utiliza la función `torch.device()` y el método `.to()` de los objetos `Tensor`.

**Ejemplo**

Se implementan operaciones en la GPU, para ello se define un tensor vacio `x`

In [ ]:
x = torch.empty((2,2))

se modifica `x` de manera inplace en la CPU

In [ ]:
x.add_(1)

Se define un flujo en la CPU

In [ ]:
# Se detecta la presencia de una GPU
if torch.cuda.is_available():
    # Se obtiene un objeto que apunta a la GPU descubierta
    device = torch.device("cuda")          
    
    # Se crea un tensor y se carga en la GPU
    y = torch.ones_like(x, device=device)  #
    
    # Se envia un vector a la GPU luego de ser definido
    x = x.to(device)                       
    
    # Se obtiene un resultado en la GPU y se transfiere a la CPU
    z = x + y
    print('z en GPU: \n', z)
    print('z en CPU:\n', z.to("cpu", torch.double))

**Ejercicio**

1. ¿Qué notación común siguen las operaciones *in place* para tensores?

El motor central en las redes neuronales de PyTorch es la librería  `autograd`. Esta entrega herramientas de diferenciacion automatica para todas las operaciones hechas sobre objetos tipo `Tensor`. 

Como se estudio anteriormente, los tensores son objetos similares a los arreglos de NumPy en cuanto a sus métodos y manejo. Aparte de permitir el computo en GPU, poseen el atributo  el atributo booleano `.requires_grad`. Si para un tensor, tal atributo tiene el valor `True`, se comienzan a registrar todas operaciones aplicadas aplicadas sobre este. Tal registro se lleva a cabo de manera a automática, generando una estructura jeraquica codificada en un grafo denotado como DCG (grafo dinámico computacional). Este grafo es grafo acíclico y dirigido, sus hojas son en efecto, los tensores siendo seguidos (*input*) y las raíces son los tensores de respuesta luego de la última operación (*output*). Tal grafo permite calcular gradientes siguiendo un esquema similar al de de backpropagation, donde se multiplican gradientes en cada etapa desde la raíz al cierto nodo hoja de interés.

Para generar el registro antes descrito, las operaciones realizadas sobre un tensor input son asociadas a una operación por medio de la clase `Function`. Cada tensor del cual se lleva un registro, tendrá el atributo `.grad_fn` que referencia al objeto de clase `Function` que dio origien a tal tensor. En el caso de tensores creados explícitamente, `.grad_fn`será `None`. 

**Ejemplo**

Se crea un grafo usando tensores que requieren gradiente.

In [ ]:
x = torch.tensor(5.0, requires_grad=True)
y = torch.tensor(1.0)
z = x * y + 1

# Componentes del grafo
for i, name in zip([x, y, z], "XYZ"):

    print(f"{name}\n Datos: {i.data}\n requires_grad: {i.requires_grad}\n\
 grad: {i.grad}\n grad_fn: {i.grad_fn}\n\n ¿Es hoja? : {i.is_leaf}\n")

Para hacer operaciones sin registrar gradientes se utiliza el context manager `torch.no_grad()`.

In [ ]:
x = torch.tensor(25.0, requires_grad=True)
y = x * 2

print('Se requiere gradiente en y:', y.requires_grad)

with torch.no_grad():
    y = x * 2
    print('Se requiere gradiente en y (context manager): ', y.requires_grad)

**Ejercicio**

1. ¿Qué función tiene el método `.detach()` en un tensor?

El método `.backward()` calcula los gradientes al aplicarse sobre un **valor escalar** (tensor unitario) es,to se hace recorriendo el grafo generado por las operaciones siguiendo el esquema de propagación hacia atrás. Los gradientes calculados se guardan en el atributo `.grad` de cada nodo hoja. 

**Ejemplo**

Se estudia el gradiente de una función 

In [ ]:
# Se genera el grafo de manera dinamica en cada operacion
x = torch.tensor(5.0, requires_grad = True)
z = x ** 3

# Se calculan los gradientes
z.backward() 

En el caso anterior, se tiene la función $z = x^3$ cuyo gradiente con respecto a $x$ es $3x^2$ que evaluado en $x=5$ pasa a ser $75$. Se confirma lo anterior

In [ ]:
print(x.grad.data)

Cuando se llama `z.backward()` se pasa como argumento el tensor unitario `torch.tensor(5.0)`, es decir, se calcula `z.backward(torch.tensor(5.0))`, en este caso, dicho argumento es el *gradiente externo* utilizado para terminar las multiplicaciones asociadas a la regla de la cadena. Por ejemplo, si se tienen los tensores :

In [ ]:
x = torch.tensor([2.0, 4.0, 8.0], requires_grad = True)
y = torch.tensor([1.0 , 3.0 , 5.0], requires_grad = True)

z = x*y
z

In [ ]:
z.backward(torch.tensor([1.0,1.0,1.0]))
x.grad

Entonces, para calcular los gradientes de `z` con respecto a `x` o a `y`, se debe entregar un gradiente externo de dimensión 3. 

En términos generales, si se tiene una función vectorial $\vec{y}=f(\vec{x})$, entonces el gradiente de $\vec{y}$ con respecto a $\vec{x}$ serpa la matriz Jacobiana:
$$
J=\left(\begin{array}{ccc}\frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}\end{array}\right)
$$

Para efectuar el algoritmo de propagación hacia atrás, `torch.autograd` calcula productos entre matrices jacobianas y vectores, por lo que dado un vector $v=\left(v_{1} , v_{2} , \ldots , v_{m}\right)^{T}$, se calcula $v^{T} \cdot J$. Si en tal caso, $v$ es el gradiente de una función escalar $l=g(\vec{y})$, es decir, $v=\left( \frac{\partial l}{\partial y_{1}} , \cdots , \frac{\partial l}{\partial y_{m}}\right)^{T}$, entonces por la regla de la cadena, el producto siguiente
$$
J^{T} \cdot v=\left(\begin{array}{ccc}\frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}\end{array}\right)\left(\begin{array}{c}\frac{\partial l}{\partial y_{1}} \\ \vdots \\ \frac{\partial l}{\partial y_{m}}\end{array}\right)=\left(\begin{array}{c}\frac{\partial l}{\partial x_{1}} \\ \vdots \\ \frac{\partial l}{\partial x_{n}}\end{array}\right)
$$

Pasa a ser el gradiente de $l$ con respecto a $\vec{x}$. Tal desarrollo hace que tenga sentido entregar gradientes externos a una cadena de operaciones sin outpus escalares.


**Ejercicio**

1. Estudie la documentación de la clase `Function` perteneciente al módulo `torch.autograd`. [link](https://pytorch.org/docs/stable/autograd.html#function). 

## Aprendizaje Profundo

El aprendizaje profundo o *deep learning* consiste en la aplicación de modelos basados en redes neuronales con múltiples capas ocultas. Este tipo de redes presenta la ventaja pues sus estructuras no solo permiten predecir el output de un modelo, sino que también aprender característcas en los datos, es decir, generar representaciones abstractas de los datos de entrada. 

En la práctica, los algoritmos de aprendizaje profundo son redes neuronales consistentes de neuronas y capas. Para diferenciarlos, es necesario observar la arquitectura y sus procesos de aprendizaje. Acá podemos distinguir a grandes rasgos las redes MLP o feedforward, convolucionales, Recurrentes y autoencoders. 

Antes de pasar a estudiar dichas arquitecturas, se discutirán algunos aspectos referentes al proceso de aprendizaje que en este tipo de algoritmos se relaciona estrechamente con los métodos de optimización a utilizar. 

Ya se aplicó el algoritmo de **descenso de gradiente estocástico SGD** en combinación con backpropagation. Este algoritmo puede ser modificado al introducir **momentum**. El algorimto SGD se puede resumir de la siguiente manera:

* Se sigue la dinámica de actualización $w \rightarrow w-\lambda \nabla(J(w))$, donde $\lambda$ se denota como *learning rate*. 

* Para añadir momentum, se puede añadir un *peso de actualización* de la forma:
$$
\triangle w \rightarrow \mu \cdot \triangle w-\lambda(\nabla J(w))
$$

Donde $\triangle w$ representa el vector de actualización añadido a $w$ en la iteración anterior, $\mu$ es un parámetro que determina la dependencia de nuevos valores para $w$ en función valores anteriores. $\mu \cdot \triangle w$ se denota como *momentum*. 

* Finalmente se crea una nueva regla de actualización mediante:
$$
w \rightarrow w+\Delta w
$$

**Ejercicio**

1. ¿Qué ventajas puede proporcionar añadir la componente de momentum?

2. Otro algoritmo útil es **Adam**, investigue sobre su funcionamiento y ventajas. [link](https://arxiv.org/abs/1412.6980)

### Redes Neuronales Convolucionales

Las redes neuronales convolucionales CNN (por sus siglas en ingles) fueron concebidas en el contexto de *visión computacional*. Aquí se busca obtener conocimiento por medio del manejo de representaciones interpretables de manera óptica por humanos (imagenes y videos por ejemplo). Tal proceso de obtención de información posee características bastante especificas y difíciles de codificar en una máquina. Por ejemplo, en el contexto de imagenes, se puede asumir que píxeles cercanos (bajo cierta métrica) se relacionan de mejor manera y que por tanto su aglomeración tiene sentido, por otra parte, el significado de una imagen puede depender fuertemente del contexto de ciertas estructuras o patrones abstractos difíciles de programar en un software. 

Una red neuronal múlticapa, en su forma más sencilla, considera a los datos de entrada como arreglos y a priori no saca provecho de la estructura de su input. En el ejemplo de las imagenes, tal red no puede distinguir vecinos posicionales (píxeles cercanos) pues recibe como input un arreglo unidimensional. En este contexto, nacen las CNN's, estas permiten sacar provecho de la estructura (especialmente en imágenes) haciendo posible mejorar la interacción entre neuronas cercanas. En problemas visuales, esto consiste en hacer que las neuronas procesen información originada en píxeles cercanos entre si, para lograr esto se hace uso de **capas convolucionales**.

Una capa convolucional consiste de un conjunto de filtros o *kernels*. Estos consisten en un conjunto de pesos (a aprender), cada kernel es aplicado en *areas* de los datos de entrada. En la siguiente animación ([fuente](https://m-alcu.github.io/)) se puede observar un kernel de 3x3x3 aplicado sobre una imagen de 9x9x3, en ambos casos, el último índidce es 3 y corresponde a la cantidad de canales de color asociados a la imagen procesada.

![conv_net](https://m-alcu.github.io/assets/cntk103d_conv2d_final.gif)

Si la imagen es la capa input de la red, la acción del kernel consiste en asociar cada dato de entrada con un peso, en el ejemplo de la animación, el kernel aplicado posee 3x3x3 pesos que se *comparten* en cada región procesada de la imagen. El output de esta aplicación es una suma ponderada entre los píxeles de entrada y los pesos del filtro. Al recorrer toda la imagen, se tiene un nuevo conjunto de inputs sobre los cuales es posible operar con una función de activación u otra capa de neuronas (convolcional o simplemente lineal-conectada).

La idea intuitiva del kernel el ser capas de extraer alguna caracteística especifica de la imagen input (contornos, agrupaciones de colores, formas, etc...).  Gracias a que los pesos del kernel se mantienen mientras recorre la imagen, se logra una reducción en la cantidad de parámetros necesarios para entrenar una capa de la red. En algunas oportunidades, los kernels entrenados (conjuntos de pesos optimizados según alguna función de pérdida) pueden ser interpretables.

**Ejercicio**

1. Considere que se tiene una imagen input de dimensiones (*l*,*a*) y se aplica un filtro de tamaño (*fl*,*fa*). ¿ Cual es la dimensión del arreglo resultante? ¿Qué ocurre si se agrega profundidad de colores?

En términos generales, se dice que los filtros convolcionales operan sobre *volumenes de entrada*, esto permite abstraer el modelo a contextos más generales que problemas visuales. 

Una **Capa convolucional** consiste en un conjunto de filtros aplicados sobre un volumen de entrada, los volúmenes de salida de cada filtro conforman el output de la capa. Para tener una idea de la cantidad de parámetros asociados.

**Ejemplo**

Se construye un filtro y se aplica sobre una imagen. En primera instancia se define la función `conv` que toma una imagen y un filtro, luego opera el filtro sobre la imagen. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm


def conv(image, im_filter):
    # input dims
    height = image.shape[0]
    width = image.shape[1]
    # output dims
    im_c = np.zeros((height - len(im_filter) + 1, width - len(im_filter) + 1))

    # itera sobre la imagen
    for row in range(len(im_c)):
        for col in range(len(im_c[0])):

            # aplica el filtro
            for i in range(len(im_filter)):
                for j in range(len(im_filter[0])):

                    im_c[row, col] += image[row + i, col + j] * im_filter[i][j]

    # valores borde
    im_c[im_c > 255] = 255
    im_c[im_c < 0] = 0
    
    # obtiene imagenes

    plt.figure()
    plt.imshow(image, cmap=cm.Greys_r)
    plt.show()

    plt.imshow(im_c, cmap=cm.Greys_r)
    plt.show()

In [ ]:
from PIL import Image

#Carga la imagen
image_rgb = np.asarray(
    Image.open('/home/nico/Escritorio/P3_example.jpg').convert("RGB"))

# transforma a escala de grises
image_grayscale = np.mean(image_rgb, axis=2, dtype=np.uint)

In [ ]:
#filtro blurr
blur = np.full([10, 10], 1. / 100)
conv(image_grayscale, blur)

## Stride 'padding
## Regularizción 
## Data augmentation
## Cargar redes pre entrenadas
## Autoencoders
## Redes Recurrentes
## RNN's
 
